In [1]:
from config import set_path

set_path()

import sys
print(sys.version)


3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]


In [2]:
from core import load_data
from core.process_text import process_training_data

[nltk_data] Downloading package punkt to /home/h-pgy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/h-pgy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = load_data('parquet')

In [4]:
data = process_training_data(data)

Categorias com menos que o minino: ['Esporte e Lazer', 'Inovação e Tecnologia', 'Pessoa com Deficiência']


In [5]:
data.sample(5)

,excerto,categoria,processed_text
2110,Revisar - Essa intensificação seria representa...,Transportes e Mobilidade,intensificacao representar numero
1111,Também foi aprovado para esse ano cinco metas ...,Outros temas,tambem aprovar ano cinco meta aqui orcamento i...
512,Também trago aqui a nossa grande luta para tra...,Saúde,tambem tragar aqui grande luta trazer unidade ...
1285,Já faz alguns anos que as nossas metas vêm cai...,Outros temas,ja fazer algum ano meta vir cair valor arrecad...
3717,"Uma outra questão, questão dos idosos, são 75 ...",Saúde,outro questao questao idoso sao mil idoso pinh...


In [6]:
data['categoria'].value_counts()

categoria
Saúde                                     546
Outros temas                              508
Transportes e Mobilidade                  472
Zeladoria Urbana e melhorias de bairro    462
Desenvolvimento Econômico e Trabalho      385
Assistência Social                        289
Meio Ambiente                             278
Habitação                                 239
Educação                                  211
Cultura                                   157
Segurança Urbana                          141
Direitos Humanos e Cidadania              129
Infraestrutura Urbana e Obras             124
Urbanismo e Licenciamento                 121
Name: count, dtype: int64

In [7]:
data.sample(5)

,excerto,categoria,processed_text
2977,Programa Contrata SP - Pessoa 60+. Criar e rea...,Desenvolvimento Econômico e Trabalho,programa contratar sp pessoa criar realizar ed...
1303,Nós abordamos sempre na questão de trazer nova...,Educação,abordar sempre questao trazer novo creche aqui...
3234,Impedir o aumento do déficit habitacional. Ent...,Habitação,impedir aumento deficit habitacional entregar ...
3326,"Fiscalização de poluição de cheiro, fumaça e r...",Zeladoria Urbana e melhorias de bairro,fiscalizacao poluicao cheiro fumaca ruir comer...
367,"E da saúde, também a ampliação da UBS do Jardi...",Saúde,saude tambem ampliacao ubs jardim apurar ubs n...


In [8]:
data['excerto'].duplicated().any()

np.False_

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.pipeline import Pipeline

In [10]:
# Separar features e labels
X_train, X_test, y_train, y_test = train_test_split(data['processed_text'], data['categoria'], test_size=0.2, random_state=42)

# Criar pipeline com TF-IDF e SVC
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=3, max_df=0.85, ngram_range=(1,2))),
    ('svc', SVC(kernel='linear', random_state=42))
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Avaliação
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


                                        precision    recall  f1-score   support

                    Assistência Social       0.75      0.78      0.76        60
                               Cultura       0.86      0.71      0.77        34
  Desenvolvimento Econômico e Trabalho       0.76      0.73      0.75        89
          Direitos Humanos e Cidadania       0.69      0.47      0.56        19
                              Educação       0.82      0.75      0.79        44
                             Habitação       0.81      0.74      0.77        34
         Infraestrutura Urbana e Obras       0.64      0.35      0.45        26
                         Meio Ambiente       0.75      0.76      0.76        51
                          Outros temas       0.54      0.74      0.62        99
                                 Saúde       0.79      0.80      0.79       103
                      Segurança Urbana       0.82      0.47      0.60        19
              Transportes e Mobilidade 

In [17]:
len(X_train)

3249

In [11]:
from sklearn.model_selection import GridSearchCV

# Definir os hiperparâmetros a serem testados
param_grid = {
    'svc__C': [0.01, 0.1, 1, 10, 100],  # Regularização
    'svc__kernel': ['linear', 'rbf', 'poly'],  # Tipos de kernel
    'svc__gamma': [1e-3, 1e-2, 0.1, 1],  # Parâmetro usado para kernels não lineares
    'svc__class_weight': ['balanced', None]
}

# Criar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Executar a busca
grid_search.fit(X_train, y_train)





Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.0s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=poly; total time=   4.1s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.1s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=rbf; total time=   4.1s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.3s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.3s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=rbf; total time=   4.3s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.4s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=rbf; tota

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_df=0.85, min_df=3,
                                                        ngram_range=(1, 2))),
                                       ('svc',
                                        SVC(kernel='linear',
                                            random_state=42))]),
             n_jobs=-1,
             param_grid={'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__class_weight': ['balanced', None],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=2)

In [12]:
# Melhor combinação de hiperparâmetros
print("Melhores parâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

Melhores parâmetros: {'svc__C': 10, 'svc__class_weight': 'balanced', 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}
Melhor acurácia: 0.7178


In [13]:

# Avaliação no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy no teste: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 no teste: {f1_score(y_test, y_pred, average='micro'):.4f}")

                                        precision    recall  f1-score   support

                    Assistência Social       0.71      0.75      0.73        60
                               Cultura       0.84      0.76      0.80        34
  Desenvolvimento Econômico e Trabalho       0.78      0.72      0.75        89
          Direitos Humanos e Cidadania       0.48      0.58      0.52        19
                              Educação       0.77      0.77      0.77        44
                             Habitação       0.79      0.76      0.78        34
         Infraestrutura Urbana e Obras       0.53      0.35      0.42        26
                         Meio Ambiente       0.67      0.76      0.72        51
                          Outros temas       0.55      0.68      0.61        99
                                 Saúde       0.81      0.76      0.78       103
                      Segurança Urbana       0.82      0.47      0.60        19
              Transportes e Mobilidade 

In [14]:
print(f"F1 no teste com peso por classe: {f1_score(y_test, y_pred, average='weighted'):.4f}")

F1 no teste com peso por classe: 0.7165


In [18]:
import joblib
import os
from config import DATA_DIR 

# Salvar o modelo treinado
joblib.dump(best_model, os.path.join(DATA_DIR, 'svc_model.pkl'))
print("Modelo salvo com sucesso!")


Modelo salvo com sucesso!
